In [1]:
from legisph.senate.website import SenateWebsite
senate = SenateWebsite()
bills = list(filter(lambda b: not isinstance(b, Exception), senate.fetch_all_bills()))

In [2]:
import polars as pl
from itertools import chain

all_statuses = [
    *[b.legislative_status for b in bills],
    *list(chain(*[b.legislative_history for b in bills])),
]
[
    s.item.split("\n")
    for s in all_statuses 
    if s.item.find("\n") > 0
]

[['Letter from Senate Secretary Myra Marie D. Villarica, transmitting nine (9) enrolled',
  'copies of the consolidated version of SBN-2522 and HBN-6522, for the signature of Speaker Lord Allan Jay',
  'Q. Velasco and Secretary General Mark Llandro L. Mendoza;'],
 ['Enrolled copies of SBN-2506 and HBN-9459 sent to the Office of the',
  'President through PLLO for the signature of the President of the Philippines, His Excellency, President',
  'Rodrigo Roa Duterte;'],
 ['Letter from Senate Secretary Myra Marie D. Villarica, transmitting nine (9) enrolled',
  'copies of SBN-2504 and HBN-10015 for the signature of Speaker Lord Allan Jay Q. Velasco and Secretary General Mark',
  'Llandro L. Mendoza;'],
 ['Enrolled copies of SBN-2504 and HBN-10015 sent to the Office of the',
  'President through PLLO for the signature of the President of the Philippines, His Excellency, President',
  'Rodrigo Roa Duterte;'],
 ['Letter from Senate Secretary Myra Marie D. Villarica, dated June 21, 2022, trans

In [12]:
from legisph.senate.bill_status import * 

unparsed_statuses = [
    status
    for status in all_statuses
    if len(
        parse_senate_bill_status(
            status,
            classes=[
                PendingInCommittee,
                JointProceedings,
                Introduced,
                CommitteeReportCalendaredForOrdinaryBusiness,
                ConsolidatedOrSubstitutedInCommitteeReport,
                TechnicalWorkingGroup,
                FirstReading,
                CommitteeProceedings,
                ApprovedOnSecondReading,
                TransferredToCalendarForSpecialOrder,
                PeriodOfInterpellationClosed,
                SponsorshipSpeech,
                SponsorshipSpeechInsertedIntoRecord,
                ApprovedOnThirdReading,
                SentToHouseOfRepresentatives,
                IndividualAmendmentsOpened,
                IndividualAmendmentsClosed,
                CommitteeAmendmentsOpened,
                CommitteeAmendmentsClosed,
                CopiesDistributed,
                ConsolidatedWithApprovedBill,
                ApprovedByPresident
            ],
        )
    )
    == 0
]
status_df = pl.DataFrame({"status": [status.item for status in unparsed_statuses]})
list(
    (
        status_df.groupby("status")
            .agg([pl.count()])
            .sort("count", reverse=True)
    )["status"]
)
# (
#     status_df.groupby("status")
#             .agg([pl.count()])
#             .sort("count", reverse=True)
#             .filter(pl.col("count") > 10)
# )

['Submitted said Committee Report to the Senate;',
 'Pending Second Reading, Special Order',
 'Abstention:  (None) ;',
 'Against:  (None) ;',
 'Conference Committee Report Approved by the Senate;',
 'Against:         N o n e;',
 'Printed Copies were distributed to the Senators;',
 'Transferred from Ordinary Business to Special Order;',
 'Abstention:    N o n e;',
 'Period of amendment closed;',
 'Sponsor: Senator FRANCIS "CHIZ" G. ESCUDERO;',
 'Abstention:   N o n e;',
 'Pending Second Reading, Ordinary Business',
 'Pending in the House of Representatives',
 'Sent to the Archives',
 'Against:  N o n e ;',
 'SENT TO THE ARCHIVES.',
 'Abstention:  N o n e ;',
 'Interpellation of Senator FRANKLIN M. DRILON;',
 'Assigned for Consideration by Special Order;',
 'Conducted ORGANIZATIONAL MEETING/BRIEFING;',
 'Motion of the Majority Leader, Senator Alan Peter Compañero S. Cayetano, that the matters appearing in the Reference of Business for todays session be deemed read and transmitted to the 

In [ ]:
from legisph.senate.bill_status import * 
from legisph.senate.models import SenateBill
import datetime
sponsorship_speech = set([
    s.item
    for s in all_statuses
    if s.item.lower().startswith("sponsorship speech")
])
unparsed_statuses = [
    status
    for status in sponsorship_speech
    if SponsorshipSpeech.parse(SenateBill.SenateBillStatus(
        date = datetime.date.today(),
        item = status
    ))[0] is None
]
unparsed_statuses

['Sponsorship speech inserted into the record;',
 'Sponsorship speech on the conference committee report of Senator(s) M. A. MADRIGAL, upon her request the Joint Explanation of the Conference Committee Report was adopted as the sponsorship speech;',
 'Sponsorship speech of Senators VICENTE C. SOTTO III and RAMON BONG REVILLA JR.;',
 'Sponsorship speech on Senator EDGARDO J. ANGARA on the Conference Committee Report;',
 'Sponsorship speech on the conference committee report of Senator(s) JOKER P. ARROYO;',
 'Sponsorship speech on the conference committee report of Senator(s) JUAN MIGUEL F. ZUBIRI;',
 'Sponsorship speech on the conference committee report of Senator(s) EDGARDO J. ANGARA;',
 'Sponsorship speech on the Conference Committee Report of Senator(s) RICHARD J. GORDON;',
 'Sponsorship speech on the conference committee report of Senator(s) RICHARD "DICK" J. GORDON;',
 'Sponsorship speech of Senators JUAN MIGUEL "MIGZ" F. ZUBIRI;',
 'Sponsorship speech o Senator FRANCIS "CHIZ" G. 